<a href="https://colab.research.google.com/github/Neha-kumari31/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/LS_DS_431_RNN_and_LSTM_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [0]:
# TODO - Words, words, mere words, no matter from the heart.
#Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) 
#to generate, and returns generated text of that size.

In [3]:
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

import tensorflow as tf

from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import os
import textwrap

# fix random seed for reproducibility
np.random.seed(45)

Using TensorFlow backend.


In [4]:
url = "https://www.gutenberg.org/files/100/100-0.txt"
path_to_file = tf.keras.utils.get_file('shakespeare.txt', url)
raw_text = open(path_to_file, 'r', encoding='utf-8').read()

5783552/5777367 [==============================] - 6s 1us/step


In [5]:
print(raw_text[:1500])
print(f'Length of the text: {len(raw_text)}')

﻿
Project Gutenberg’s The Complete Works of William Shakespeare, by William
Shakespeare

This eBook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever.  You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this eBook or online at
www.gutenberg.org.  If you are not located in the United States, you’ll
have to check the laws of the country where you are located before using
this ebook.


Title: The Complete Works of William Shakespeare

Author: William Shakespeare

Release Date: January 1994 [EBook #100]
Last Updated: November 7, 2019

Language: English

Character set encoding: UTF-8

*** START OF THIS PROJECT GUTENBERG EBOOK THE COMPLETE WORKS OF WILLIAM SHAKESPEARE ***




The Complete Works of William Shakespeare



by William Shakespeare




      Contents



               THE SONNETS

               ALL’S WELL THAT ENDS WELL

             

In [6]:
def pre_process(text):
    text = text[900:-21100]
    text = text.replace("\r", "")
    text = " ".join(text.split())
    print (f'Length of text: {len(text)} characters long')
    return text
text = pre_process(raw_text)
text[:100]

Length of text: 5258116 characters long


'Contents THE SONNETS ALL’S WELL THAT ENDS WELL THE TRAGEDY OF ANTONY AND CLEOPATRA AS YOU LIKE IT TH'

In [0]:
# create mapping of unique chars to integers
chars = sorted(set(raw_text))

char_int = {c:i for i, c in enumerate(chars)}
int_char = {i:c for i, c in enumerate(chars)}

In [8]:
len(chars)

107

In [9]:
chars

['\t',
 '\n',
 ' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 '@',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 '\\',
 ']',
 '_',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '|',
 '}',
 'Æ',
 'É',
 'à',
 'â',
 'æ',
 'ç',
 'è',
 'é',
 'ê',
 'î',
 'œ',
 '—',
 '‘',
 '’',
 '“',
 '”',
 '\ufeff']

In [10]:
# Create the sequence data
maxlen = 40
step = 5

encoded = [char_int[c] for c in raw_text]

sequences = [] # Each element is 40 chars long
next_char = [] # One element for each sequence

for i in range(0, len(encoded) - maxlen, step):
    
    sequences.append(encoded[i : i + maxlen])
    next_char.append(encoded[i + maxlen])
    
print('sequences: ', len(sequences))

sequences:  1114623


In [11]:
sequences[0]

[106,
 1,
 46,
 79,
 76,
 71,
 66,
 64,
 81,
 2,
 37,
 82,
 81,
 66,
 75,
 63,
 66,
 79,
 68,
 103,
 80,
 2,
 50,
 69,
 66,
 2,
 33,
 76,
 74,
 77,
 73,
 66,
 81,
 66,
 2,
 53,
 76,
 79,
 72,
 80]

In [0]:
# Create x & y
x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences),len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
        
    y[i, next_char[i]] = 1

In [13]:
# Number of sequences, sequence length, unique charactersin that sequence
x.shape, y.shape

((1114623, 40, 107), (1114623, 107))

In [14]:
len(chars)

107

###Create Model

In [0]:
def create_model1(X, y):
    # Define the LSTM model
    model = Sequential()
    model.add(LSTM(256, input_shape=(x.shape[1], x.shape[2])))
    model.add(Dropout(0.2))
    model.add(Dense(y.shape[1], activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model


model1 = create_model1(x, y)

In [0]:
def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / 1
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [0]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    
    generated = ''
    
    # sentence = text[start_index: start_index + maxlen]
    sentence = 'How much mother love her kids '
    generated += sentence
    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_int[char]] = 1
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_char = int_char[next_index]
        
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [18]:
# fit the model
model1.fit(x, y,
          batch_size=32,
          epochs=25,
          callbacks=[print_callback])

Epoch 1/25
34830/34832 [============================>.] - ETA: 0s - loss: 1.9019
----- Generating text after Epoch: 0


NameError: ignored

###Add Callback

In [0]:
!pip install wandb

In [0]:
import wandb
from wandb.keras import WandbCallback

wandb.init(project="angus")

# define the checkpoint
# Directory where the checkpoints will be saved
checkpoint_directory = './training_checkpoints'

if not os.path.exists(checkpoint_directory ):
    os.makedirs(checkpoint_directory)
    print(f'New directory created at $:->{checkpoint_directory}')
file_name ="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint_filename = os.path.join(checkpoint_directory, file_name)
checkpoint_callback = ModelCheckpoint(checkpoint_filename, monitor='loss', verbose=1, save_best_only=True, mode='min')

callbacks_list = [checkpoint_callback, WandbCallback()]

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN